# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7ff260da2ee0>
├── 'a' --> tensor([[-0.1467,  1.4538,  0.1681],
│                   [ 0.3379,  0.5104, -2.0302]])
└── 'x' --> <FastTreeValue 0x7ff260da2f40>
    └── 'c' --> tensor([[ 0.3295,  2.4345,  1.1758,  1.9668],
                        [ 0.3509, -1.0403,  1.8030,  0.9048],
                        [-1.2673, -0.1461,  0.4934, -0.9372]])

In [4]:
t.a

tensor([[-0.1467,  1.4538,  0.1681],
        [ 0.3379,  0.5104, -2.0302]])

In [5]:
%timeit t.a

67.4 ns ± 0.0514 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7ff260da2ee0>
├── 'a' --> tensor([[ 1.0341, -0.0723,  0.5965],
│                   [-1.1801, -1.8352, -0.0924]])
└── 'x' --> <FastTreeValue 0x7ff260da2f40>
    └── 'c' --> tensor([[ 0.3295,  2.4345,  1.1758,  1.9668],
                        [ 0.3509, -1.0403,  1.8030,  0.9048],
                        [-1.2673, -0.1461,  0.4934, -0.9372]])

In [7]:
%timeit t.a = new_value

68.5 ns ± 0.075 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.1467,  1.4538,  0.1681],
               [ 0.3379,  0.5104, -2.0302]]),
    x: Batch(
           c: tensor([[ 0.3295,  2.4345,  1.1758,  1.9668],
                      [ 0.3509, -1.0403,  1.8030,  0.9048],
                      [-1.2673, -0.1461,  0.4934, -0.9372]]),
       ),
)

In [10]:
b.a

tensor([[-0.1467,  1.4538,  0.1681],
        [ 0.3379,  0.5104, -2.0302]])

In [11]:
%timeit b.a

61.1 ns ± 0.0136 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.2825,  0.2289, -1.2486],
               [-1.4071,  2.3314,  0.8199]]),
    x: Batch(
           c: tensor([[ 0.3295,  2.4345,  1.1758,  1.9668],
                      [ 0.3509, -1.0403,  1.8030,  0.9048],
                      [-1.2673, -0.1461,  0.4934, -0.9372]]),
       ),
)

In [13]:
%timeit b.a = new_value

506 ns ± 0.1 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

816 ns ± 1.33 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

10.9 µs ± 25.8 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

142 µs ± 384 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

143 µs ± 598 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7ff260da2d60>
├── 'a' --> tensor([[[-0.1467,  1.4538,  0.1681],
│                    [ 0.3379,  0.5104, -2.0302]],
│           
│                   [[-0.1467,  1.4538,  0.1681],
│                    [ 0.3379,  0.5104, -2.0302]],
│           
│                   [[-0.1467,  1.4538,  0.1681],
│                    [ 0.3379,  0.5104, -2.0302]],
│           
│                   [[-0.1467,  1.4538,  0.1681],
│                    [ 0.3379,  0.5104, -2.0302]],
│           
│                   [[-0.1467,  1.4538,  0.1681],
│                    [ 0.3379,  0.5104, -2.0302]],
│           
│                   [[-0.1467,  1.4538,  0.1681],
│                    [ 0.3379,  0.5104, -2.0302]],
│           
│                   [[-0.1467,  1.4538,  0.1681],
│                    [ 0.3379,  0.5104, -2.0302]],
│           
│                   [[-0.1467,  1.4538,  0.1681],
│                    [ 0.3379,  0.5104, -2.0302]]])
└── 'x' --> <FastTreeValue 0x7ff260da2d30>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

31.8 µs ± 73.2 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7ff1a902b7f0>
├── 'a' --> tensor([[-0.1467,  1.4538,  0.1681],
│                   [ 0.3379,  0.5104, -2.0302],
│                   [-0.1467,  1.4538,  0.1681],
│                   [ 0.3379,  0.5104, -2.0302],
│                   [-0.1467,  1.4538,  0.1681],
│                   [ 0.3379,  0.5104, -2.0302],
│                   [-0.1467,  1.4538,  0.1681],
│                   [ 0.3379,  0.5104, -2.0302],
│                   [-0.1467,  1.4538,  0.1681],
│                   [ 0.3379,  0.5104, -2.0302],
│                   [-0.1467,  1.4538,  0.1681],
│                   [ 0.3379,  0.5104, -2.0302],
│                   [-0.1467,  1.4538,  0.1681],
│                   [ 0.3379,  0.5104, -2.0302],
│                   [-0.1467,  1.4538,  0.1681],
│                   [ 0.3379,  0.5104, -2.0302]])
└── 'x' --> <FastTreeValue 0x7ff23c280a60>
    └── 'c' --> tensor([[ 0.3295,  2.4345,  1.1758,  1.9668],
                        [ 0.3509, -1.0403,  1.8030,  0.9048],
                 

In [23]:
%timeit t_cat(trees)

30.3 µs ± 99.4 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

57 µs ± 239 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.1467,  1.4538,  0.1681],
                [ 0.3379,  0.5104, -2.0302]],
       
               [[-0.1467,  1.4538,  0.1681],
                [ 0.3379,  0.5104, -2.0302]],
       
               [[-0.1467,  1.4538,  0.1681],
                [ 0.3379,  0.5104, -2.0302]],
       
               [[-0.1467,  1.4538,  0.1681],
                [ 0.3379,  0.5104, -2.0302]],
       
               [[-0.1467,  1.4538,  0.1681],
                [ 0.3379,  0.5104, -2.0302]],
       
               [[-0.1467,  1.4538,  0.1681],
                [ 0.3379,  0.5104, -2.0302]],
       
               [[-0.1467,  1.4538,  0.1681],
                [ 0.3379,  0.5104, -2.0302]],
       
               [[-0.1467,  1.4538,  0.1681],
                [ 0.3379,  0.5104, -2.0302]]]),
    x: Batch(
           c: tensor([[[ 0.3295,  2.4345,  1.1758,  1.9668],
                       [ 0.3509, -1.0403,  1.8030,  0.9048],
                       [-1.2673, -0.1461,  0.4934, -0.9372]],
         

In [26]:
%timeit Batch.stack(batches)

79 µs ± 73.7 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.1467,  1.4538,  0.1681],
               [ 0.3379,  0.5104, -2.0302],
               [-0.1467,  1.4538,  0.1681],
               [ 0.3379,  0.5104, -2.0302],
               [-0.1467,  1.4538,  0.1681],
               [ 0.3379,  0.5104, -2.0302],
               [-0.1467,  1.4538,  0.1681],
               [ 0.3379,  0.5104, -2.0302],
               [-0.1467,  1.4538,  0.1681],
               [ 0.3379,  0.5104, -2.0302],
               [-0.1467,  1.4538,  0.1681],
               [ 0.3379,  0.5104, -2.0302],
               [-0.1467,  1.4538,  0.1681],
               [ 0.3379,  0.5104, -2.0302],
               [-0.1467,  1.4538,  0.1681],
               [ 0.3379,  0.5104, -2.0302]]),
    x: Batch(
           c: tensor([[ 0.3295,  2.4345,  1.1758,  1.9668],
                      [ 0.3509, -1.0403,  1.8030,  0.9048],
                      [-1.2673, -0.1461,  0.4934, -0.9372],
                      [ 0.3295,  2.4345,  1.1758,  1.9668],
                      [ 0.3509, -

In [28]:
%timeit Batch.cat(batches)

166 µs ± 376 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

323 µs ± 1.07 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
